In [1]:
!pip install nnsight
!pip install msgspec "python-socketio[client]"
!pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24/24 [nnsight]1/24 [python-engineio]


In [2]:
import torch
import io
import base64
import csv
from PIL import Image
from diffusers import StableDiffusionPipeline
from nnsight.modeling.diffusion import DiffusionModel
import pandas as pd
from nnsight import NNsight
import numpy as np

/opt/conda/envs/py_3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


In [3]:
MODEL_PATH = "stabilityai/stable-diffusion-2-1-base"
SEED = 17 # Do not modify the seed as per challenge rules

# Initialize the model
model = DiffusionModel(
    MODEL_PATH,
    dispatch=True
).to("cuda")

# Function to get MLP activations from the Text Encoder
def get_MLP_activations(prompt):
    LAYERS = model.text_encoder.text_model.encoder.layers
    saved_activations = []

    with model.generate(
        prompt,
        seed=SEED
    ):
        for layer in LAYERS:
            saved_activations.append(layer.mlp.output.save())

    actual_activations = [s.value for s in saved_activations]
    layers_tensor = torch.stack(actual_activations, dim=0)
    return layers_tensor

Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  4.73it/s]


In [4]:
def pil_image_to_string(img: Image.Image) -> str:
    """
    Convert a PIL image to a Base64-encoded string.

    Args:
        img: PIL.Image object.

    Returns:
        Base64 string representing the image.
    """
    buf = io.BytesIO()
    img.save(buf, format='PNG') # PNG for lossless compression
    # FIX: Corrected typo from base66 to base64
    return base64.b64encode(buf.getvalue()).decode('utf-8')


In [ ]:
# --- REFINED AND EXPANDED Prompt Pairs for Winter/Summer Concept Isolation ---
# These pairs aim for greater diversity and stricter negation to produce a purer steering vector.
prompt_pairs = [
    # --- Landscapes & Natural Environments ---
    ("An aerial view of a snow-covered mountain range, icy peaks, frozen lake, stark white landscape",
     "An aerial view of a lush green mountain range, rocky peaks, clear lake, vibrant green landscape"),

    ("A lone pine tree heavily laden with snow, in a silent, freezing forest, under a grey, winter sky",
     "A lone pine tree surrounded by wildflowers, in a vibrant, sunlit forest, under a clear, summer sky"),

    ("A frozen waterfall cascading down an icy cliff, with a thin layer of snow on the rocks",
     "A gushing waterfall cascading down a mossy cliff, with lush greenery and clear water"),

    ("A wide shot of an abandoned lighthouse on a windswept, snow-dusted coastline, stormy grey ocean",
     "A wide shot of an abandoned lighthouse on a sunny, sandy coastline, clear blue ocean"),

    ("A vast arctic tundra covered in fresh powder snow, ice sheet, polar bears in distance",
     "A vast green savannah with tall grass, thorny trees, elephants in distance"),

    ("A calm river completely frozen over, skaters on the ice, snow banks on the sides",
     "A calm river flowing gently, kayakers on the water, green banks on the sides"),

    ("A quiet forest path blanketed in fresh, untouched snow, cold silence, bare branches",
     "A quiet forest path lined with vibrant green moss, chirping birds, leafy canopies"),

    ("Jagged icebergs floating in a dark, frigid sea, surrounded by vast, cold water",
     "Lush tropical islands floating in a bright, warm sea, surrounded by clear, blue water"),

    ("A field of frosted winter wheat under a pale sun, covered in hoarfrost",
     "A field of golden summer wheat under a bright sun, swaying in a warm breeze"),

    # --- Urban & Architectural Scenes ---
    ("A bustling city street at dusk, covered in fresh snow, people bundled in thick coats, steaming breath",
     "A bustling city street at dusk, bathed in warm light, people in light clothing, outdoor cafes"),

    ("An old stone bridge covered in thick frost and icicles, leading to a snowy village",
     "An old stone bridge covered in moss and vines, leading to a sunny village"),

    ("A public park covered in a blanket of snow, with bare trees and frozen fountains",
     "A public park with lush green lawns, blooming flowerbeds, and splashing fountains"),

    # --- Objects & Details ---
    ("A close-up of delicate snowflakes falling on an old wooden fence, frosty branches in the background",
     "A close-up of dew drops on lush green leaves, blooming roses in the background, warm sunlight"),

    ("A steaming mug of hot chocolate on a frosty window sill, condensation on glass",
     "A glass of iced tea with lemon on a sunlit window sill, clear glass"),

    ("A thick woolen scarf and mittens hanging on a snow-dusted branch",
     "A light linen scarf and straw hat hanging on a leafy branch"),

    ("Crystal clear ice cubes rattling in a chilled glass, condensation",
     "Warm water gently steaming in a ceramic mug"),

    # --- People & Activities ---
    ("Children building a snowman in a suburban park, surrounded by snowdrifts, bare trees",
     "Children playing on a swing set in a suburban park, surrounded by green grass, leafy trees"),

    ("A cozy cabin interior with a roaring fireplace, looking out at a heavy snowfall, frosted windowpanes",
     "A cozy cabin interior with open windows, looking out at a sunny meadow, blooming flowers"),

    ("A person dressed in a heavy parka walking through a blizzard, face hidden by hood",
     "A person dressed in light shorts and t-shirt walking on a sunny beach, sunglasses on"),

    ("Skiers descending a powdery mountain slope, snow kicking up behind them",
     "Hikers ascending a grassy mountain trail, wildflowers in bloom"),

    ("People huddled around a roaring bonfire in a snowy landscape, bundled up",
     "People having a lively barbecue in a grassy park, wearing light clothes"),

    ("A group ice skating on a frozen lake, rosy cheeks from the cold, winter sports",
     "A group swimming in a clear lake, tanned skin from the sun, summer fun"),

    # --- Abstract & Atmospheric Elements ---
    ("The crisp, sharp air of a winter morning, cold breath visible, white light",
     "The warm, humid air of a summer morning, gentle breeze, golden light"),

    ("A landscape dominated by shades of white, grey, and deep blue, cold colors, muted tones",
     "A landscape dominated by vibrant greens, yellows, and warm reds, lively colors, bright hues"),

    ("A feeling of quiet stillness and introspection, muffled by snow, serene atmosphere",
     "A feeling of vibrant energy and outdoor activity, buzzing with life, joyful atmosphere")
]


In [ ]:

prompt_pairs_with_weights = [
    ("An aerial view of a snow-covered mountain range, icy peaks, frozen lake, stark white landscape", "An aerial view of a lush green mountain range, rocky peaks, clear lake, vibrant green landscape", 1.0),
    ("A lone pine tree heavily laden with snow, in a silent, freezing forest, under a grey, winter sky", "A lone pine tree surrounded by wildflowers, in a vibrant, sunlit forest, under a clear, summer sky", 0.95),
    ("A frozen waterfall cascading down an icy cliff, with a thin layer of snow on the rocks", "A gushing waterfall cascading down a mossy cliff, with lush greenery and clear water", 0.95),
    ("A wide shot of an abandoned lighthouse on a windswept, snow-dusted coastline, stormy grey ocean", "A wide shot of an abandoned lighthouse on a sunny, sandy coastline, clear blue ocean", 0.85),
    ("A vast arctic tundra covered in fresh powder snow, ice sheet, polar bears in distance", "A vast green savannah with tall grass, thorny trees, elephants in distance", 1.0),
    ("A calm river completely frozen over, skaters on the ice, snow banks on the sides", "A calm river flowing gently, kayakers on the water, green banks on the sides", 0.95),
    ("A quiet forest path blanketed in fresh, untouched snow, cold silence, bare branches", "A quiet forest path lined with vibrant green moss, chirping birds, leafy canopies", 0.9),
    ("Jagged icebergs floating in a dark, frigid sea, surrounded by vast, cold water", "Lush tropical islands floating in a bright, warm sea, surrounded by clear, blue water", 1.0),
    ("A field of frosted winter wheat under a pale sun, covered in hoarfrost", "A field of golden summer wheat under a bright sun, swaying in a warm breeze", 0.9),
    ("A bustling city street at dusk, covered in fresh snow, people bundled in thick coats, steaming breath", "A bustling city street at dusk, bathed in warm light, people in light clothing, outdoor cafes", 0.95),
    ("An old stone bridge covered in thick frost and icicles, leading to a snowy village", "An old stone bridge covered in moss and vines, leading to a sunny village", 0.9),
    ("A public park covered in a blanket of snow, with bare trees and frozen fountains", "A public park with lush green lawns, blooming flowerbeds, and splashing fountains", 0.9),
    ("A close-up of delicate snowflakes falling on an old wooden fence, frosty branches in the background", "A close-up of dew drops on lush green leaves, blooming roses in the background, warm sunlight", 0.95),
    ("A steaming mug of hot chocolate on a frosty window sill, condensation on glass", "A glass of iced tea with lemon on a sunlit window sill, clear glass", 0.9),
    ("A thick woolen scarf and mittens hanging on a snow-dusted branch", "A light linen scarf and straw hat hanging on a leafy branch", 0.85),
    ("Crystal clear ice cubes rattling in a chilled glass, condensation", "Warm water gently steaming in a ceramic mug", 0.7),
    ("Children building a snowman in a suburban park, surrounded by snowdrifts, bare trees", "Children playing on a swing set in a suburban park, surrounded by green grass, leafy trees", 1.0),
    ("A cozy cabin interior with a roaring fireplace, looking out at a heavy snowfall, frosted windowpanes", "A cozy cabin interior with open windows, looking out at a sunny meadow, blooming flowers", 0.95),
    ("A person dressed in a heavy parka walking through a blizzard, face hidden by hood", "A person dressed in light shorts and t-shirt walking on a sunny beach, sunglasses on", 0.95),
    ("Skiers descending a powdery mountain slope, snow kicking up behind them", "Hikers ascending a grassy mountain trail, wildflowers in bloom", 1.0),
    ("People huddled around a roaring bonfire in a snowy landscape, bundled up", "People having a lively barbecue in a grassy park, wearing light clothes", 0.9),
    ("A group ice skating on a frozen lake, rosy cheeks from the cold, winter sports", "A group swimming in a clear lake, tanned skin from the sun, summer fun", 0.95),
    ("The crisp, sharp air of a winter morning, cold breath visible, white light", "The warm, humid air of a summer morning, gentle breeze, golden light", 1.0),
    ("A landscape dominated by shades of white, grey, and deep blue, cold colors, muted tones", "A landscape dominated by vibrant greens, yellows, and warm reds, lively colors, bright hues", 1.0),
    ("A feeling of quiet stillness and introspection, muffled by snow, serene atmosphere", "A feeling of vibrant energy and outdoor activity, buzzing with life, joyful atmosphere", 0.95)
]


In [ ]:
# --- Compute NORMALIZED Steering Vector using all prompt pairs ---
print("Computing steering vector using multiple prompt pairs...")

sum_of_unnormalized_steering_vectors = None

for i, (positive_prompt, negative_prompt) in enumerate(prompt_pairs):
    print(f"Processing prompt pair {i+1}/{len(prompt_pairs)}: Positive='{positive_prompt}', Negative='{negative_prompt}'")

    activations_positive = get_MLP_activations(positive_prompt)
    activations_negative = get_MLP_activations(negative_prompt)

    current_unnormalized_steering_vector = activations_positive - activations_negative

    if sum_of_unnormalized_steering_vectors is None:
        sum_of_unnormalized_steering_vectors = current_unnormalized_steering_vector
    else:
        sum_of_unnormalized_steering_vectors += current_unnormalized_steering_vector

print("Normalizing the accumulated steering vector...")
steering_vector = torch.nn.functional.normalize(sum_of_unnormalized_steering_vectors, p=2, dim=-1)
print(f"Final steering vector calculated with shape: {steering_vector.shape}")

Weighted Steering

In [ ]:
import torch
import gc

# =================================================================
# 1. COMBINED AND WEIGHTED PROMPT PAIRS
# =================================================================
# Combined and expanded list of prompt pairs with weights.
prompt_pairs_with_weights = [
    # --- Original Set ---
    ("An aerial view of a snow-covered mountain range, icy peaks, frozen lake, stark white landscape", "An aerial view of a lush green mountain range, rocky peaks, clear lake, vibrant green landscape", 1.0),
    ("A lone pine tree heavily laden with snow, in a silent, freezing forest, under a grey, winter sky", "A lone pine tree surrounded by wildflowers, in a vibrant, sunlit forest, under a clear, summer sky", 0.95),
    ("A frozen waterfall cascading down an icy cliff, with a thin layer of snow on the rocks", "A gushing waterfall cascading down a mossy cliff, with lush greenery and clear water", 0.95),
    ("A wide shot of an abandoned lighthouse on a windswept, snow-dusted coastline, stormy grey ocean", "A wide shot of an abandoned lighthouse on a sunny, sandy coastline, clear blue ocean", 0.85),
    ("A vast arctic tundra covered in fresh powder snow, ice sheet, polar bears in distance", "A vast green savannah with tall grass, thorny trees, elephants in distance", 1.0),
    ("A calm river completely frozen over, skaters on the ice, snow banks on the sides", "A calm river flowing gently, kayakers on the water, green banks on the sides", 0.95),
    ("A quiet forest path blanketed in fresh, untouched snow, cold silence, bare branches", "A quiet forest path lined with vibrant green moss, chirping birds, leafy canopies", 0.9),
    ("Jagged icebergs floating in a dark, frigid sea, surrounded by vast, cold water", "Lush tropical islands floating in a bright, warm sea, surrounded by clear, blue water", 1.0),
    ("A field of frosted winter wheat under a pale sun, covered in hoarfrost", "A field of golden summer wheat under a bright sun, swaying in a warm breeze", 0.9),
    ("A bustling city street at dusk, covered in fresh snow, people bundled in thick coats, steaming breath", "A bustling city street at dusk, bathed in warm light, people in light clothing, outdoor cafes", 0.95),
    ("An old stone bridge covered in thick frost and icicles, leading to a snowy village", "An old stone bridge covered in moss and vines, leading to a sunny village", 0.9),
    ("A public park covered in a blanket of snow, with bare trees and frozen fountains", "A public park with lush green lawns, blooming flowerbeds, and splashing fountains", 0.9),
    ("A close-up of delicate snowflakes falling on an old wooden fence, frosty branches in the background", "A close-up of dew drops on lush green leaves, blooming roses in the background, warm sunlight", 0.95),
    ("A steaming mug of hot chocolate on a frosty window sill, condensation on glass", "A glass of iced tea with lemon on a sunlit window sill, clear glass", 0.9),
    ("A thick woolen scarf and mittens hanging on a snow-dusted branch", "A light linen scarf and straw hat hanging on a leafy branch", 0.85),
    ("Crystal clear ice cubes rattling in a chilled glass, condensation", "Warm water gently steaming in a ceramic mug", 0.7),
    ("Children building a snowman in a suburban park, surrounded by snowdrifts, bare trees", "Children playing on a swing set in a suburban park, surrounded by green grass, leafy trees", 1.0),
    ("A cozy cabin interior with a roaring fireplace, looking out at a heavy snowfall, frosted windowpanes", "A cozy cabin interior with open windows, looking out at a sunny meadow, blooming flowers", 0.95),
    ("A person dressed in a heavy parka walking through a blizzard, face hidden by hood", "A person dressed in light shorts and t-shirt walking on a sunny beach, sunglasses on", 0.95),
    ("Skiers descending a powdery mountain slope, snow kicking up behind them", "Hikers ascending a grassy mountain trail, wildflowers in bloom", 1.0),
    ("People huddled around a roaring bonfire in a snowy landscape, bundled up", "People having a lively barbecue in a grassy park, wearing light clothes", 0.9),
    ("A group ice skating on a frozen lake, rosy cheeks from the cold, winter sports", "A group swimming in a clear lake, tanned skin from the sun, summer fun", 0.95),
    ("The crisp, sharp air of a winter morning, cold breath visible, white light", "The warm, humid air of a summer morning, gentle breeze, golden light", 1.0),
    ("A landscape dominated by shades of white, grey, and deep blue, cold colors, muted tones", "A landscape dominated by vibrant greens, yellows, and warm reds, lively colors, bright hues", 1.0),
    ("A feeling of quiet stillness and introspection, muffled by snow, serene atmosphere", "A feeling of vibrant energy and outdoor activity, buzzing with life, joyful atmosphere", 0.95),

    # --- New Abstract Pair Added ---
    ("A blurry, abstract close-up of rough, grey concrete texture, devoid of any organic forms or vibrant colors.", "A vibrant, detailed macro shot of a delicate flower petal, showing intricate textures and vivid colors, soft focus background.", 1.0)
]

# =================================================================
# 2. COMPUTE WEIGHTED STEERING VECTOR
# =================================================================
# This function should already be defined in your notebook
# def get_MLP_activations(prompt):
#     ...

print("Computing weighted steering vector...")

# Initialize a tensor to accumulate the weighted vectors
initial_activations = get_MLP_activations(prompt_pairs_with_weights[0][0])
sum_of_weighted_vectors = torch.zeros_like(initial_activations, device='cpu')

# Loop through each prompt pair and its weight
for i, (positive_prompt, negative_prompt, weight) in enumerate(prompt_pairs_with_weights):
    print(f"Processing prompt pair {i+1}/{len(prompt_pairs_with_weights)} with weight {weight}...")

    # Get activations for both prompts in the pair
    activations_positive = get_MLP_activations(positive_prompt)
    activations_negative = get_MLP_activations(negative_prompt)

    # Calculate the steering vector for this specific pair
    # --- THIS IS THE FIX ---
    # Move the result to the CPU before doing any math with the CPU sum tensor
    current_steering_vector = (activations_positive - activations_negative).to('cpu')

    # Apply the weight to this vector and add it to the total sum
    sum_of_weighted_vectors += current_steering_vector * weight

print("\nNormalizing the accumulated steering vector...")

# Normalize the final vector to have a unit length
steering_vector = torch.nn.functional.normalize(sum_of_weighted_vectors, p=2, dim=-1)

print(f"Final weighted steering vector calculated with shape: {steering_vector.shape}")


Computing weighted steering vector...


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:07<00:00,  6.86it/s]


Processing prompt pair 1/26 with weight 1.0...


100%|██████████| 50/50 [00:07<00:00,  6.89it/s]


...

Processing prompt pair 25/26 with weight 0.95...


100%|██████████| 50/50 [00:07<00:00,  6.87it/s]


Processing prompt pair 26/26 with weight 1.0...


100%|██████████| 50/50 [00:07<00:00,  6.84it/s]



Normalizing the accumulated steering vector...
Final weighted steering vector calculated with shape: torch.Size([23, 1, 77, 1024])


Save steering vector

In [ ]:
import numpy as np
import torch

# Assume 'steering_vector' is your calculated tensor

print("Saving steering vector to steering_vector.txt...")

# 1. Move the tensor to the CPU and convert it to a NumPy array
steering_vector_np = steering_vector.cpu().numpy()

# 2. The tensor is likely 3D (layers, tokens, dimensions).
#    We need to reshape it to 2D to save it as a standard text file.
#    This flattens the last two dimensions into one.
num_layers = steering_vector_np.shape[0]
reshaped_vector = steering_vector_np.reshape(num_layers, -1)

# 3. Save the reshaped NumPy array to a text file
np.savetxt('steering_vector_weigthed_new.txt', reshaped_vector, fmt='%.18e')

print("File 'steering_vector.txt' saved successfully.")

Saving steering vector to steering_vector.txt...
File 'steering_vector.txt' saved successfully.


In [ ]:
# producing original images # DON'T RERUNIT, ALREADY THERE
import os
# Load the target prompts from prompts.csv
prompts_df = pd.read_csv('prompts.csv')

# Directory to save original images for comparison
ORIGINAL_IMAGES_DIR = "original_images"
os.makedirs(ORIGINAL_IMAGES_DIR, exist_ok=True) # Create the directory if it doesn't exist

# Prepare a list to store the results for submission (steered images only for DEFAULT_EPS_FOR_SUBMISSION)
submission_data = []

print("Generating original images and multiple sets of steered images for different eps values...")

# Generate original images once
print("\n--- Generating ORIGINAL images ---")
for index, row in prompts_df.iterrows():
    target_prompt = row['prompt']
    prompt_id = row['ID']
    print(f"  Generating ORIGINAL image for ID {prompt_id}: '{target_prompt}'")
    with model.generate(target_prompt, seed=SEED): # Removed 'as model_generate'
        original_image = model.output.images[0].save() 
    original_image_path = os.path.join(ORIGINAL_IMAGES_DIR, f"original_prompt_{prompt_id}.png")
    original_image.save(original_image_path)
    print(f"  Original image saved to '{original_image_path}'")

In [59]:
import os
import pandas as pd

# Initialize an empty list to store the submission data.
# This ensures you start with a clean list every time you run the cell.
submission_data = []

# Iterate through different eps values THESE ARE THE ONES THAT PERFORMED BETTER
DEFUALT_EPS = 2.5

eps_values = [2.5]
for eps_value in eps_values:
    print(f"\n--- Generating STEERED images for eps = {eps_value} ---")
    STEERED_IMAGES_DIR = f"steered_images_eps_{eps_value}"
    os.makedirs(STEERED_IMAGES_DIR, exist_ok=True)

    for index, row in prompts_df.iterrows():
        target_prompt = row['prompt']
        prompt_id = row['ID']

        # Always add/enhance the winter concept as per previous clarification
        # This applied_steering_vector_full still contains all layers
        applied_steering_vector_full = steering_vector * eps_value
        steering_type = f"injecting/enhancing winter with eps={eps_value}"

        print(f"  Processing ID {prompt_id}: '{target_prompt}' ({steering_type})...")
        with model.generate(target_prompt, seed=SEED):
            # FIX: Iterate with enumerate to get layer index and select the correct slice
            for layer_idx, layer in enumerate(model.text_encoder.text_model.encoder.layers):
                # Select the steering vector slice corresponding to the current layer
                steering_vector_for_this_layer = applied_steering_vector_full[layer_idx]
                layer.mlp.output = layer.mlp.output + steering_vector_for_this_layer.to(layer.mlp.output.device)
            steered_image = model.output.images[0].save()

        # Save the steered image to the eps-specific directory
        steered_image_path = os.path.join(STEERED_IMAGES_DIR, f"steered_prompt_{prompt_id}.png")
        steered_image.save(steered_image_path)
        print(f"  Steered image saved to '{steered_image_path}'")

        # If this is the default eps for submission, add to submission_data
        if eps_value == DEFUALT_EPS:
            img_base64 = pil_image_to_string(steered_image)
            submission_data.append({'ID': prompt_id, 'images': img_base64})

# Create the final DataFrame from the data you already generated
submission_df = pd.DataFrame(submission_data)

# Save the DataFrame to submission.csv
submission_df.to_csv('submission_weight_more_2.3.csv', index=False)

print("submission_weight.csv file created successfully!")


--- Generating STEERED images for eps = 2.5 ---
  Processing ID 0: 'A family having a picnic in a sunny summer meadow' (injecting/enhancing winter with eps=2.5)...


100%|██████████| 50/50 [00:07<00:00,  6.75it/s]


  Steered image saved to 'steered_images_eps_2.5/steered_prompt_0.png'
  Processing ID 1: 'A close-up of ice cream melting on a hot summer day' (injecting/enhancing winter with eps=2.5)...


100%|██████████| 50/50 [00:07<00:00,  6.82it/s]


  Steered image saved to 'steered_images_eps_2.5/steered_prompt_1.png'
  Processing ID 2: 'A vibrant summer flower garden with butterflies' (injecting/enhancing winter with eps=2.5)...


...

  Steered image saved to 'steered_images_eps_2.5/steered_prompt_27.png'
  Processing ID 28: 'A beach volleyball game under a clear blue sky' (injecting/enhancing winter with eps=2.5)...


100%|██████████| 50/50 [00:07<00:00,  6.86it/s]


  Steered image saved to 'steered_images_eps_2.5/steered_prompt_28.png'
  Processing ID 29: 'A summer rain shower over a quiet country road' (injecting/enhancing winter with eps=2.5)...


100%|██████████| 50/50 [00:07<00:00,  6.87it/s]


  Steered image saved to 'steered_images_eps_2.5/steered_prompt_29.png'
submission_weight.csv file created successfully!


In [ ]:
# second trial: visual comparison
import os
from PIL import Image
import pandas as pd
import io
import base64
from IPython.display import display, HTML

# Redefine pil_image_to_string here for this standalone plotting block
# This function is necessary to convert PIL Images to Base64 for HTML display
def pil_image_to_string(img: Image.Image) -> str:
    """
    Convert a PIL image to a Base64-encoded string.

    Args:
        img: PIL.Image object.

    Returns:
        Base64 string representing the image.
    """
    buf = io.BytesIO()
    img.save(buf, format='PNG')
    return base64.b64encode(buf.getvalue()).decode('utf-8')

# Define directories and parameters (these should match your main script)
ORIGINAL_IMAGES_DIR = "original_images"
# Ensure these eps_values match what you used in your generation script
eps_values = [1.9,2.0]

print("\n--- Displaying generated images for visual comparison ---")

# Load prompts.csv again to get prompt IDs and text for labeling
try:
    # Attempt to read prompts.csv from the current directory
    prompts_df = pd.read_csv("/kaggle/input/imgprompts/prompts.csv")
except FileNotFoundError:
    print("Error: prompts.csv not found for plotting. Please ensure the file is in the same directory or update the path.")
    print("Cannot proceed with plotting without prompt information.")
    prompts_df = pd.DataFrame() # Create an empty DataFrame to prevent further errors

if not prompts_df.empty:
    for index, row in prompts_df.iterrows():
        target_prompt = row['prompt']
        prompt_id = row['ID']

        # Start HTML table for this prompt
        html_output = f"<h3 style='margin-top:20px;'>Prompt ID: {prompt_id}</h3>"
        html_output += f"<p>Prompt: <b>'{target_prompt}'</b></p>"
        html_output += "<table style='width:100%; border-collapse: collapse; table-layout: fixed;'>"
        html_output += "<tr>"
        html_output += "<th style='width:20%; border: 1px solid #ddd; padding: 8px; text-align: center;'>Original Image</th>"
        for eps_value in eps_values:
            html_output += f"<th style='width:20%; border: 1px solid #ddd; padding: 8px; text-align: center;'>Steered (eps={eps_value})</th>"
        html_output += "</tr><tr>"

        # Load and display original image
        original_image_path = os.path.join(ORIGINAL_IMAGES_DIR, f"original_prompt_{prompt_id}.png")
        try:
            original_image = Image.open(original_image_path)
            original_img_base64 = pil_image_to_string(original_image)
        except FileNotFoundError:
            original_img_base64 = "" # Placeholder if image not found
            print(f"Warning: Original image not found for ID {prompt_id} at '{original_image_path}'. Skipping display for this image.")

        html_output += f"<td style='border: 1px solid #ddd; padding: 8px; text-align: center;'>"
        if original_img_base64:
            html_output += f"<img src='data:image/png;base64,{original_img_base64}' width='200'>"
        else:
            html_output += "Image not found"
        html_output += "</td>"

        # Load and display steered images for this prompt across all eps values
        for eps_value in eps_values:
            STEERED_IMAGES_DIR = f"steered_images_eps_{eps_value}"
            steered_image_path = os.path.join(STEERED_IMAGES_DIR, f"steered_prompt_{prompt_id}.png")

            try:
                steered_image = Image.open(steered_image_path)
                steered_img_base64 = pil_image_to_string(steered_image)
            except FileNotFoundError:
                steered_img_base64 = "" # Placeholder if image not found
                print(f"Warning: Steered image (eps={eps_value}) not found for ID {prompt_id} at '{steered_image_path}'. Skipping display for this image.")

            html_output += f"<td style='border: 1px solid #ddd; padding: 8px; text-align: center;'>"
            if steered_img_base64:
                html_output += f"<img src='data:image/png;base64,{steered_img_base64}' width='200'>"
            else:
                html_output += "Image not found"
            html_output += "</td>"

        html_output += "</tr></table><hr style='margin-bottom:30px;'>" # End row, end table, add horizontal rule for separation
        display(HTML(html_output)) # Display the table for this prompt in the notebook output

print("\n--- Visual comparison display complete. ---")
print("Please review the images above to decide on the best 'eps_value' for your submission.")
print("Once decided, update the 'DEFAULT_EPS_FOR_SUBMISSION' variable in your main script and re-run it to generate the final submission.csv.")

# --- END OF NEW CODE BLOCK F

K-Top version

In [36]:
import torch
import gc

# =================================================================
# 1. SETUP: Define prompt pairs and helper function
# =================================================================
# This list should be defined in your notebook.
# Each item is a tuple: (positive_prompt, negative_prompt, weight)
prompt_pairs_with_weights = [
    ("An aerial view of a snow-covered mountain range, icy peaks, frozen lake, stark white landscape", "An aerial view of a lush green mountain range, rocky peaks, clear lake, vibrant green landscape", 1.0),
    ("A lone pine tree heavily laden with snow, in a silent, freezing forest, under a grey, winter sky", "A lone pine tree surrounded by wildflowers, in a vibrant, sunlit forest, under a clear, summer sky", 0.95),
    ("A frozen waterfall cascading down an icy cliff, with a thin layer of snow on the rocks", "A gushing waterfall cascading down a mossy cliff, with lush greenery and clear water", 0.95),
    ("A wide shot of an abandoned lighthouse on a windswept, snow-dusted coastline, stormy grey ocean", "A wide shot of an abandoned lighthouse on a sunny, sandy coastline, clear blue ocean", 0.85),
    ("A vast arctic tundra covered in fresh powder snow, ice sheet, polar bears in distance", "A vast green savannah with tall grass, thorny trees, elephants in distance", 1.0),
    ("A calm river completely frozen over, skaters on the ice, snow banks on the sides", "A calm river flowing gently, kayakers on the water, green banks on the sides", 0.95),
    ("A quiet forest path blanketed in fresh, untouched snow, cold silence, bare branches", "A quiet forest path lined with vibrant green moss, chirping birds, leafy canopies", 0.9),
    ("Jagged icebergs floating in a dark, frigid sea, surrounded by vast, cold water", "Lush tropical islands floating in a bright, warm sea, surrounded by clear, blue water", 1.0),
    ("A field of frosted winter wheat under a pale sun, covered in hoarfrost", "A field of golden summer wheat under a bright sun, swaying in a warm breeze", 0.9),
    ("A bustling city street at dusk, covered in fresh snow, people bundled in thick coats, steaming breath", "A bustling city street at dusk, bathed in warm light, people in light clothing, outdoor cafes", 0.95),
    ("An old stone bridge covered in thick frost and icicles, leading to a snowy village", "An old stone bridge covered in moss and vines, leading to a sunny village", 0.9),
    ("A public park covered in a blanket of snow, with bare trees and frozen fountains", "A public park with lush green lawns, blooming flowerbeds, and splashing fountains", 0.9),
    ("A close-up of delicate snowflakes falling on an old wooden fence, frosty branches in the background", "A close-up of dew drops on lush green leaves, blooming roses in the background, warm sunlight", 0.95),
    ("A steaming mug of hot chocolate on a frosty window sill, condensation on glass", "A glass of iced tea with lemon on a sunlit window sill, clear glass", 0.9),
    ("A thick woolen scarf and mittens hanging on a snow-dusted branch", "A light linen scarf and straw hat hanging on a leafy branch", 0.85),
    ("Crystal clear ice cubes rattling in a chilled glass, condensation", "Warm water gently steaming in a ceramic mug", 0.7),
    ("Children building a snowman in a suburban park, surrounded by snowdrifts, bare trees", "Children playing on a swing set in a suburban park, surrounded by green grass, leafy trees", 1.0),
    ("A cozy cabin interior with a roaring fireplace, looking out at a heavy snowfall, frosted windowpanes", "A cozy cabin interior with open windows, looking out at a sunny meadow, blooming flowers", 0.95),
    ("A person dressed in a heavy parka walking through a blizzard, face hidden by hood", "A person dressed in light shorts and t-shirt walking on a sunny beach, sunglasses on", 0.95),
    ("Skiers descending a powdery mountain slope, snow kicking up behind them", "Hikers ascending a grassy mountain trail, wildflowers in bloom", 1.0),
    ("People huddled around a roaring bonfire in a snowy landscape, bundled up", "People having a lively barbecue in a grassy park, wearing light clothes", 0.9),
    ("A group ice skating on a frozen lake, rosy cheeks from the cold, winter sports", "A group swimming in a clear lake, tanned skin from the sun, summer fun", 0.95),
    ("The crisp, sharp air of a winter morning, cold breath visible, white light", "The warm, humid air of a summer morning, gentle breeze, golden light", 1.0),
    ("A landscape dominated by shades of white, grey, and deep blue, cold colors, muted tones", "A landscape dominated by vibrant greens, yellows, and warm reds, lively colors, bright hues", 1.0),
    ("A feeling of quiet stillness and introspection, muffled by snow, serene atmosphere", "A feeling of vibrant energy and outdoor activity, buzzing with life, joyful atmosphere", 0.95)
]

# Function to get MLP activations from the Text Encoder
def get_MLP_activations(prompt):
    LAYERS = model.text_encoder.text_model.encoder.layers
    saved_activations = []

    with model.generate(
        prompt,
        seed=SEED
    ):
        for layer in LAYERS:
            saved_activations.append(layer.mlp.output.save())

    actual_activations = [s.value for s in saved_activations]
    layers_tensor = torch.stack(actual_activations, dim=0)
    return layers_tensor

# =================================================================
# 2. CALCULATE ALL INDIVIDUAL STEERING VECTORS
# =================================================================
print("--- Calculating all individual steering vectors ---")

all_steering_vectors = []
for i, (positive_prompt, negative_prompt, weight) in enumerate(prompt_pairs_with_weights):
    print(f"Processing prompt pair {i+1}/{len(prompt_pairs_with_weights)}...")
    
    activations_positive = get_MLP_activations(positive_prompt)
    activations_negative = get_MLP_activations(negative_prompt)
    
    # Calculate the steering vector for this pair and apply its weight
    current_steering_vector = (activations_positive - activations_negative).to('cpu') * weight
    all_steering_vectors.append(current_steering_vector)

print(f"\nCalculated {len(all_steering_vectors)} individual steering vectors.")

# =================================================================
# 3. REFINE SELECTION (Top-k or Threshold)
# =================================================================

# --- Parameters for experimentation ---
SELECTION_METHOD = 'top-k'  # Change to 'threshold' or 'all'
K_VALUE = 15                # Use if SELECTION_METHOD is 'top-k'. How many of the best vectors to use.
MAGNITUDE_THRESHOLD = 50.0  # Use if SELECTION_METHOD is 'threshold'.

# --- Calculate the magnitude (strength) of each vector ---
# We use the L2 norm as a measure of each vector's power
vector_magnitudes = [torch.linalg.norm(vec) for vec in all_steering_vectors]

# --- Apply the chosen selection method ---
if SELECTION_METHOD == 'top-k':
    print(f"\n--- Applying Top-K Selection (k={K_VALUE}) ---")
    # Combine vectors with their magnitudes and sort them
    vectors_with_magnitudes = sorted(zip(all_steering_vectors, vector_magnitudes), key=lambda x: x[1], reverse=True)
    # Select the top k vectors
    selected_vectors = [vec for vec, mag in vectors_with_magnitudes[:K_VALUE]]
    print(f"Selected the top {len(selected_vectors)} vectors.")

elif SELECTION_METHOD == 'threshold':
    print(f"\n--- Applying Threshold Selection (threshold={MAGNITUDE_THRESHOLD}) ---")
    selected_vectors = []
    for vec, mag in zip(all_steering_vectors, vector_magnitudes):
        if mag > MAGNITUDE_THRESHOLD:
            selected_vectors.append(vec)
    print(f"Selected {len(selected_vectors)} vectors that passed the threshold.")
    if not selected_vectors:
        print("Warning: No vectors passed the threshold. The final steering vector will be zero. Try a lower threshold.")
        selected_vectors = [torch.zeros_like(all_steering_vectors[0])] # Avoid crashing

else:
    print("\n--- Using all vectors (no selection) ---")
    selected_vectors = all_steering_vectors

# =================================================================
# 4. COMPUTE FINAL STEERING VECTOR
# =================================================================
# Average the selected vectors to get the final result
final_sum = torch.stack(selected_vectors, dim=0).sum(dim=0)
steering_vector = torch.nn.functional.normalize(final_sum, p=2, dim=-1)

print(f"\nFinal steering vector calculated successfully with shape: {steering_vector.shape}")


--- Calculating all individual steering vectors ---
Processing prompt pair 1/25...


100%|██████████| 50/50 [00:07<00:00,  7.11it/s]


Processing prompt pair 2/25...


100%|██████████| 50/50 [00:07<00:00,  7.10it/s]


Processing prompt pair 3/25...


...

Processing prompt pair 24/25...


100%|██████████| 50/50 [00:06<00:00,  7.18it/s]


Processing prompt pair 25/25...


100%|██████████| 50/50 [00:06<00:00,  7.19it/s]



Calculated 25 individual steering vectors.

--- Applying Top-K Selection (k=15) ---
Selected the top 15 vectors.

Final steering vector calculated successfully with shape: torch.Size([23, 1, 77, 1024])


K-Top - Compute Submission

In [42]:
import os
import pandas as pd

# Initialize an empty list to store the submission data.
# This ensures you start with a clean list every time you run the cell.
submission_data = []

# Iterate through different eps values THESE ARE THE ONES THAT PERFORMED BETTER
DEFUALT_EPS = 2.1

eps_values = [2.1] # Example eps values for tuning
for eps_value in eps_values:
    print(f"\n--- Generating STEERED images for eps = {eps_value} ---")
    STEERED_IMAGES_DIR = f"steered_images_eps_{eps_value}"
    os.makedirs(STEERED_IMAGES_DIR, exist_ok=True)

    for index, row in prompts_df.iterrows():
        target_prompt = row['prompt']
        prompt_id = row['ID']

        # Always add/enhance the winter concept as per previous clarification
        # This applied_steering_vector_full still contains all layers
        applied_steering_vector_full = steering_vector * eps_value
        steering_type = f"injecting/enhancing winter with eps={eps_value}"

        print(f"  Processing ID {prompt_id}: '{target_prompt}' ({steering_type})...")
        with model.generate(target_prompt, seed=SEED):
            # FIX: Iterate with enumerate to get layer index and select the correct slice
            for layer_idx, layer in enumerate(model.text_encoder.text_model.encoder.layers):
                # Select the steering vector slice corresponding to the current layer
                steering_vector_for_this_layer = applied_steering_vector_full[layer_idx]
                layer.mlp.output = layer.mlp.output + steering_vector_for_this_layer.to(layer.mlp.output.device)
            steered_image = model.output.images[0].save()

        # Save the steered image to the eps-specific directory
        steered_image_path = os.path.join(STEERED_IMAGES_DIR, f"steered_prompt_{prompt_id}.png")
        steered_image.save(steered_image_path)
        print(f"  Steered image saved to '{steered_image_path}'")

        # If this is the default eps for submission, add to submission_data
        if eps_value == DEFUALT_EPS:
            img_base64 = pil_image_to_string(steered_image)
            submission_data.append({'ID': prompt_id, 'images': img_base64})

# Create the final DataFrame from the data you already generated
submission_df = pd.DataFrame(submission_data)

# Save the DataFrame to submission.csv
submission_df.to_csv('submission_weight_topk.csv', index=False)

print("submission_weight.csv file created successfully!")


--- Generating STEERED images for eps = 2.1 ---
  Processing ID 0: 'A family having a picnic in a sunny summer meadow' (injecting/enhancing winter with eps=2.1)...


100%|██████████| 50/50 [00:07<00:00,  7.07it/s]


  Steered image saved to 'steered_images_eps_2.1/steered_prompt_0.png'
  Processing ID 1: 'A close-up of ice cream melting on a hot summer day' (injecting/enhancing winter with eps=2.1)...


100%|██████████| 50/50 [00:07<00:00,  6.99it/s]


  Steered image saved to 'steered_images_eps_2.1/steered_prompt_1.png'
  Processing ID 2: 'A vibrant summer flower garden with butterflies' (injecting/enhancing winter with eps=2.1)...


...

  Steered image saved to 'steered_images_eps_2.1/steered_prompt_27.png'
  Processing ID 28: 'A beach volleyball game under a clear blue sky' (injecting/enhancing winter with eps=2.1)...


100%|██████████| 50/50 [00:07<00:00,  6.96it/s]


  Steered image saved to 'steered_images_eps_2.1/steered_prompt_28.png'
  Processing ID 29: 'A summer rain shower over a quiet country road' (injecting/enhancing winter with eps=2.1)...


100%|██████████| 50/50 [00:07<00:00,  6.98it/s]


  Steered image saved to 'steered_images_eps_2.1/steered_prompt_29.png'
submission_weight.csv file created successfully!


Differentiated Layers Incremental

In [69]:
import os
import pandas as pd
import numpy as np

# =================================================================
# 1. SETUP: Define the final epsilon values per layer
# =================================================================
# Initialize an empty list to store the submission data.
# This ensures you start with a clean list every time you run the cell.
submission_data = []

# --- Parameters for generating the final epsilon values ---
num_layers = 23      # The text encoder has 23 layers (0-22)
start_eps = 1.9      # The final strength for the first layer (layer 0)
end_eps = 2.5        # The final strength for the last layer (layer 22)

# --- Generate the list of final epsilon values ---
# This list will be used directly as the strength for each layer.
final_eps_per_layer = np.linspace(start_eps, end_eps, num_layers).tolist()

# --- The default EPS for submission is now just a flag, not a value to check ---
# We will generate only one set of images for the submission.
DEFAULT_EPS_FOR_SUBMISSION = True

# Load the official prompts
prompts_df = pd.read_csv('prompts.csv')

# =================================================================
# 2. GENERATE THE FINAL SUBMISSION IMAGES
# =================================================================
print(f"\n--- Generating STEERED images with a ramp from eps={start_eps} to {end_eps} ---")
STEERED_IMAGES_DIR = "steered_images_incremental"
os.makedirs(STEERED_IMAGES_DIR, exist_ok=True)

for index, row in prompts_df.iterrows():
    target_prompt = row['prompt']
    prompt_id = row['ID']

    # This list will store the description of the steering for logging
    steering_type_description_parts = []

    print(f"  Processing ID {prompt_id}: '{target_prompt}'...")
    with model.generate(target_prompt, seed=SEED):
        # Iterate through each layer of the model's text encoder
        for layer_idx, layer in enumerate(model.text_encoder.text_model.encoder.layers):
            
            # Get the final epsilon value for the current layer directly from our list
            current_layer_eps = final_eps_per_layer[layer_idx]

            # Get the steering vector slice for the current layer
            steering_vector_for_this_layer = steering_vector[layer_idx]
            
            # Apply the steering with the correct epsilon for this layer
            layer.mlp.output = layer.mlp.output + (steering_vector_for_this_layer.to(layer.mlp.output.device) * current_layer_eps)

            # Add this layer's effective epsilon to the description for logging
            steering_type_description_parts.append(f"L{layer_idx}:eps={current_layer_eps:.2f}")

        # Construct the full steering type description for logging
        steering_type = f"injecting/enhancing winter with per-layer eps: {' '.join(steering_type_description_parts)}"
        print(f"  ({steering_type})...")
        steered_image = model.output.images[0].save()

    # Save the steered image for review
    steered_image_path = os.path.join(STEERED_IMAGES_DIR, f"steered_prompt_{prompt_id}.png")
    steered_image.save(steered_image_path)
    print(f"  Steered image saved to '{steered_image_path}'")

    # Add the data to our submission list
    img_base64 = pil_image_to_string(steered_image)
    submission_data.append({'ID': prompt_id, 'images': img_base64})

# =================================================================
# 3. CREATE AND SAVE THE SUBMISSION FILE
# =================================================================
submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission_another_incremental.csv', index=False)

print("\nsubmission_final.csv file created successfully!")



--- Generating STEERED images with a ramp from eps=1.9 to 2.5 ---
  Processing ID 0: 'A family having a picnic in a sunny summer meadow'...
  (injecting/enhancing winter with per-layer eps: L0:eps=1.90 L1:eps=1.93 L2:eps=1.95 L3:eps=1.98 L4:eps=2.01 L5:eps=2.04 L6:eps=2.06 L7:eps=2.09 L8:eps=2.12 L9:eps=2.15 L10:eps=2.17 L11:eps=2.20 L12:eps=2.23 L13:eps=2.25 L14:eps=2.28 L15:eps=2.31 L16:eps=2.34 L17:eps=2.36 L18:eps=2.39 L19:eps=2.42 L20:eps=2.45 L21:eps=2.47 L22:eps=2.50)...


100%|██████████| 50/50 [00:07<00:00,  6.77it/s]


  Steered image saved to 'steered_images_incremental/steered_prompt_0.png'
  Processing ID 1: 'A close-up of ice cream melting on a hot summer day'...
  (injecting/enhancing winter with per-layer eps: L0:eps=1.90 L1:eps=1.93 L2:eps=1.95 L3:eps=1.98 L4:eps=2.01 L5:eps=2.04 L6:eps=2.06 L7:eps=2.09 L8:eps=2.12 L9:eps=2.15 L10:eps=2.17 L11:eps=2.20 L12:eps=2.23 L13:eps=2.25 L14:eps=2.28 L15:eps=2.31 L16:eps=2.34 L17:eps=2.36 L18:eps=2.39 L19:eps=2.42 L20:eps=2.45 L21:eps=2.47 L22:eps=2.50)...


100%|██████████| 50/50 [00:07<00:00,  6.84it/s]


  Steered image saved to 'steered_images_incremental/steered_prompt_1.png'
  Processing ID 2: 'A vibrant summer flower garden with butterflies'...
  (injecting/enhancing winter with per-layer eps: L0:eps=1.90 L1:eps=1.93 L2:eps=1.95 L3:eps=1.98 L4:eps=2.01 L5:eps=2.04 L6:eps=2.06 L7:eps=2.09 L8:eps=2.12 L9:eps=2.15 L10:eps=2.17 L11:eps=2.20 L12:eps=2.23 L13:eps=2.25 L14:eps=2.28 L15:eps=2.31 L16:eps=2.34 L17:eps=2.36 L18:eps=2.39 L19:eps=2.42 L20:eps=2.45 L21:eps=2.47 L22:eps=2.50)...


...

  Steered image saved to 'steered_images_incremental/steered_prompt_27.png'
  Processing ID 28: 'A beach volleyball game under a clear blue sky'...
  (injecting/enhancing winter with per-layer eps: L0:eps=1.90 L1:eps=1.93 L2:eps=1.95 L3:eps=1.98 L4:eps=2.01 L5:eps=2.04 L6:eps=2.06 L7:eps=2.09 L8:eps=2.12 L9:eps=2.15 L10:eps=2.17 L11:eps=2.20 L12:eps=2.23 L13:eps=2.25 L14:eps=2.28 L15:eps=2.31 L16:eps=2.34 L17:eps=2.36 L18:eps=2.39 L19:eps=2.42 L20:eps=2.45 L21:eps=2.47 L22:eps=2.50)...


100%|██████████| 50/50 [00:07<00:00,  6.87it/s]


  Steered image saved to 'steered_images_incremental/steered_prompt_28.png'
  Processing ID 29: 'A summer rain shower over a quiet country road'...
  (injecting/enhancing winter with per-layer eps: L0:eps=1.90 L1:eps=1.93 L2:eps=1.95 L3:eps=1.98 L4:eps=2.01 L5:eps=2.04 L6:eps=2.06 L7:eps=2.09 L8:eps=2.12 L9:eps=2.15 L10:eps=2.17 L11:eps=2.20 L12:eps=2.23 L13:eps=2.25 L14:eps=2.28 L15:eps=2.31 L16:eps=2.34 L17:eps=2.36 L18:eps=2.39 L19:eps=2.42 L20:eps=2.45 L21:eps=2.47 L22:eps=2.50)...


100%|██████████| 50/50 [00:07<00:00,  6.88it/s]


  Steered image saved to 'steered_images_incremental/steered_prompt_29.png'

submission_final.csv file created successfully!


Differentiated Layers Decremental

In [73]:
import os
import pandas as pd
import numpy as np
import gc

# =================================================================
# 1. SETUP: Define the final epsilon values per layer
# =================================================================
# Initialize an empty list to store the submission data.
submission_data = []

# --- Parameters for generating the decremental epsilon values ---
num_layers = 23      # The text encoder has 23 layers (0-22)
start_eps = 2.5      # The final strength for the first layer (layer 0)
end_eps = 1.8        # The final strength for the last layer (layer 22)

# --- Generate the list of final epsilon values ---
# This list will now ramp down from 2.5 to 1.0.
final_eps_per_layer = np.linspace(start_eps, end_eps, num_layers).tolist()

# Load the official prompts
prompts_df = pd.read_csv('prompts.csv')

# =================================================================
# 2. GENERATE THE FINAL SUBMISSION IMAGES
# =================================================================
print(f"\n--- Generating STEERED images with a ramp from eps={start_eps} to {end_eps} ---")
STEERED_IMAGES_DIR = "steered_last_decremental"
os.makedirs(STEERED_IMAGES_DIR, exist_ok=True)

for index, row in prompts_df.iterrows():
    target_prompt = row['prompt']
    prompt_id = row['ID']

    # This list will store the description of the steering for logging
    steering_type_description_parts = []

    print(f"  Processing ID {prompt_id}: '{target_prompt}'...")
    with model.generate(target_prompt, seed=SEED):
        # Iterate through each layer of the model's text encoder
        for layer_idx, layer in enumerate(model.text_encoder.text_model.encoder.layers):
            
            # Get the final epsilon value for the current layer directly from our list
            current_layer_eps = final_eps_per_layer[layer_idx]

            # Get the steering vector slice for the current layer
            steering_vector_for_this_layer = steering_vector[layer_idx]
            
            # Apply the steering with the correct epsilon for this layer
            layer.mlp.output = layer.mlp.output + (steering_vector_for_this_layer.to(layer.mlp.output.device) * current_layer_eps)

            # Add this layer's effective epsilon to the description for logging
            steering_type_description_parts.append(f"L{layer_idx}:eps={current_layer_eps:.2f}")

        # Construct the full steering type description for logging
        steering_type = f"injecting/enhancing winter with per-layer eps: {' '.join(steering_type_description_parts)}"
        print(f"  ({steering_type})...")
        steered_image = model.output.images[0].save()

    # Save the steered image for review
    steered_image_path = os.path.join(STEERED_IMAGES_DIR, f"steered_prompt_{prompt_id}.png")
    steered_image.save(steered_image_path)
    print(f"  Steered image saved to '{steered_image_path}'")

    # Add the data to our submission list
    img_base64 = pil_image_to_string(steered_image)
    submission_data.append({'ID': prompt_id, 'images': img_base64})
    
    # Force memory cleanup
    torch.cuda.empty_cache()
    gc.collect()

# =================================================================
# 3. CREATE AND SAVE THE SUBMISSION FILE
# =================================================================
submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission_last_decremental.csv', index=False)

print("\nsubmission_final_decremental.csv file created successfully!")



--- Generating STEERED images with a ramp from eps=2.5 to 1.8 ---
  Processing ID 0: 'A family having a picnic in a sunny summer meadow'...
  (injecting/enhancing winter with per-layer eps: L0:eps=2.50 L1:eps=2.47 L2:eps=2.44 L3:eps=2.40 L4:eps=2.37 L5:eps=2.34 L6:eps=2.31 L7:eps=2.28 L8:eps=2.25 L9:eps=2.21 L10:eps=2.18 L11:eps=2.15 L12:eps=2.12 L13:eps=2.09 L14:eps=2.05 L15:eps=2.02 L16:eps=1.99 L17:eps=1.96 L18:eps=1.93 L19:eps=1.90 L20:eps=1.86 L21:eps=1.83 L22:eps=1.80)...


100%|██████████| 50/50 [00:07<00:00,  6.82it/s]


  Steered image saved to 'steered_last_decremental/steered_prompt_0.png'
  Processing ID 1: 'A close-up of ice cream melting on a hot summer day'...
  (injecting/enhancing winter with per-layer eps: L0:eps=2.50 L1:eps=2.47 L2:eps=2.44 L3:eps=2.40 L4:eps=2.37 L5:eps=2.34 L6:eps=2.31 L7:eps=2.28 L8:eps=2.25 L9:eps=2.21 L10:eps=2.18 L11:eps=2.15 L12:eps=2.12 L13:eps=2.09 L14:eps=2.05 L15:eps=2.02 L16:eps=1.99 L17:eps=1.96 L18:eps=1.93 L19:eps=1.90 L20:eps=1.86 L21:eps=1.83 L22:eps=1.80)...


100%|██████████| 50/50 [00:07<00:00,  6.85it/s]


  Steered image saved to 'steered_last_decremental/steered_prompt_1.png'
  Processing ID 2: 'A vibrant summer flower garden with butterflies'...
  (injecting/enhancing winter with per-layer eps: L0:eps=2.50 L1:eps=2.47 L2:eps=2.44 L3:eps=2.40 L4:eps=2.37 L5:eps=2.34 L6:eps=2.31 L7:eps=2.28 L8:eps=2.25 L9:eps=2.21 L10:eps=2.18 L11:eps=2.15 L12:eps=2.12 L13:eps=2.09 L14:eps=2.05 L15:eps=2.02 L16:eps=1.99 L17:eps=1.96 L18:eps=1.93 L19:eps=1.90 L20:eps=1.86 L21:eps=1.83 L22:eps=1.80)...


...

  Steered image saved to 'steered_last_decremental/steered_prompt_27.png'
  Processing ID 28: 'A beach volleyball game under a clear blue sky'...
  (injecting/enhancing winter with per-layer eps: L0:eps=2.50 L1:eps=2.47 L2:eps=2.44 L3:eps=2.40 L4:eps=2.37 L5:eps=2.34 L6:eps=2.31 L7:eps=2.28 L8:eps=2.25 L9:eps=2.21 L10:eps=2.18 L11:eps=2.15 L12:eps=2.12 L13:eps=2.09 L14:eps=2.05 L15:eps=2.02 L16:eps=1.99 L17:eps=1.96 L18:eps=1.93 L19:eps=1.90 L20:eps=1.86 L21:eps=1.83 L22:eps=1.80)...


100%|██████████| 50/50 [00:07<00:00,  6.83it/s]


  Steered image saved to 'steered_last_decremental/steered_prompt_28.png'
  Processing ID 29: 'A summer rain shower over a quiet country road'...
  (injecting/enhancing winter with per-layer eps: L0:eps=2.50 L1:eps=2.47 L2:eps=2.44 L3:eps=2.40 L4:eps=2.37 L5:eps=2.34 L6:eps=2.31 L7:eps=2.28 L8:eps=2.25 L9:eps=2.21 L10:eps=2.18 L11:eps=2.15 L12:eps=2.12 L13:eps=2.09 L14:eps=2.05 L15:eps=2.02 L16:eps=1.99 L17:eps=1.96 L18:eps=1.93 L19:eps=1.90 L20:eps=1.86 L21:eps=1.83 L22:eps=1.80)...


100%|██████████| 50/50 [00:07<00:00,  6.88it/s]


  Steered image saved to 'steered_last_decremental/steered_prompt_29.png'

submission_final_decremental.csv file created successfully!
